In [54]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score,classification_report,pairwise_distances_argmin_min,silhouette_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from preprocess_data import preprocess_data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_distances,pairwise_distances
from sklearn.cluster import KMeans
from collections import Counter
from sklearn.linear_model import SGDClassifier
from sklearn.utils.class_weight import compute_class_weight


In [55]:
# Load the data
data = preprocess_data()

data['num_genres'] = data['genre'].apply(len)
data = data[data['num_genres'] == 1]

descriptions = data['description_processed'].tolist()
genres = data['genre'].tolist()

In [106]:
# Encode the genres as numerical labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(genres)

# Load a pre-trained sentence-transformer model to convert text to embeddings
#model = SentenceTransformer('bert-base-nli-mean-tokens')
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/home/student/.cache/pypoetry/virtualenvs/dataanalysisvisualizationfiles-qa5NmZKI-py3.10/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/student/.cache/pypoetry/virtualenvs/dataanalysisvisualizationfiles-qa5NmZKI-py3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [107]:
# Convert descriptions to vector embeddings
X = model.encode(descriptions, show_progress_bar=True)

# Split the data into train and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the embeddings for cosine similarity
X_train_full_normalized = normalize(X_train_full, axis=1, norm='l2')
X_test_normalized = normalize(X_test, axis=1, norm='l2')

# Active Learning parameters
initial_train_size = 50 
iterations = 20
sample_size = 1000
uncertainty_threshold = 0.2
budget_per_iteration = 500    

#Selecting initial training set randomly
np.random.seed(42)
pool_indices = np.random.choice(len(X_train_full_normalized), initial_train_size, replace=False)
X_train = X_train_full_normalized[pool_indices]
y_train = np.array(y_train_full)[pool_indices]


Batches: 100%|██████████| 762/762 [01:05<00:00, 11.71it/s]


## Simple ML Models

In [ ]:


# Initialize the FAISS Index for Cosine Similarity
faiss.omp_set_num_threads(12)
embedding_dim = X_train_full_normalized.shape[1]
index = faiss.IndexFlatIP(embedding_dim)  # Inner product index for cosine similarity
index.add(X_train_full_normalized)  # Add all normalized vectors to the index

# Remaining pool of indices
remaining_indices = list(set(range(len(X_train_full_normalized))) - set(pool_indices))

# Initialize the classifier
#clf = RandomForestClassifier(random_state=42)
clf = SGDClassifier(loss='log_loss', random_state=42)
clf.partial_fit(X_train, y_train, classes=np.unique(y_train_full))
#clf.fit(X_train, y_train)

### Random Sampling

In [28]:
X_pool, X_train, y_pool, y_train = train_test_split(X_train_full, y_train_full, test_size=0.00255, random_state=42)

In [29]:
accuracy_scores = []


for iteration in range(iterations):
    # Incrementally train the classifier with the current training set
    clf.partial_fit(X_train, y_train)

    # Predict and evaluate performance on the test set
    y_pred = clf.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    
    print(f"Iteration {iteration + 1}: Accuracy = {accuracy_scores[-1]:.4f}")

    # Random sampling of new data points
    sample_size = min(sample_size, len(X_pool))
    selected_indices = np.random.choice(len(X_pool), size=sample_size, replace=False)
    X_sample = X_pool[selected_indices]

    # Handle potential differences in data type or format
    try:
        y_sample = y_pool[selected_indices]
    except:
        y_sample = y_pool.iloc[selected_indices]
    
    # Update the training set with new samples
    X_train = np.vstack([X_train, X_sample])
    y_train = np.concatenate([y_train, y_sample])

    # Remove the selected samples from the pool
    X_pool = np.delete(X_pool, selected_indices, axis=0)
    y_pool = np.delete(y_pool, selected_indices, axis=0)

    # Stop criteria based on the pool size or maximum training size
    if len(X_pool) == 0 or len(X_pool) < sample_size or len(X_train) >= 60000:
        break

# Print the final classification report
print(classification_report(y_test, y_pred))

Iteration 1: Accuracy = 0.2650
Iteration 2: Accuracy = 0.5822


Iteration 3: Accuracy = 0.5449
Iteration 4: Accuracy = 0.5366
Iteration 5: Accuracy = 0.6241
Iteration 6: Accuracy = 0.6358
Iteration 7: Accuracy = 0.6374
Iteration 8: Accuracy = 0.6430
Iteration 9: Accuracy = 0.6446
Iteration 10: Accuracy = 0.6303
Iteration 11: Accuracy = 0.6455
Iteration 12: Accuracy = 0.6428
Iteration 13: Accuracy = 0.6444
Iteration 14: Accuracy = 0.6506
Iteration 15: Accuracy = 0.6475
Iteration 16: Accuracy = 0.6485
Iteration 17: Accuracy = 0.6430
Iteration 18: Accuracy = 0.6422
Iteration 19: Accuracy = 0.6504
              precision    recall  f1-score   support

           0       0.56      0.36      0.44       134
           1       0.60      0.06      0.12        47
           2       0.00      0.00      0.00        19
           3       0.00      0.00      0.00        10
           4       0.63      0.60      0.62      1371
           5       0.00      0.00      0.00        35
           6       0.65      0.86      0.74      2233
           7       0.50      0

/home/student/.cache/pypoetry/virtualenvs/dataanalysisvisualizationfiles-qa5NmZKI-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/student/.cache/pypoetry/virtualenvs/dataanalysisvisualizationfiles-qa5NmZKI-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/student/.cache/pypoetry/virtualenvs/dataanalysisvisualizationfiles-qa5NmZKI-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined an

### ANN As Clustering

In [8]:
# Active Learning Loop
for iteration in range(iterations):
    # Define number of clusters
    num_clusters = min(int(np.sqrt(len(remaining_indices))), len(remaining_indices))
    
    # Initialize FAISS clustering
    clustering = faiss.Clustering(embedding_dim, num_clusters)
    clustering.verbose = False
    clustering.niter = 50  # Number of iterations for clustering
    
    # Convert remaining indices to the appropriate format
    remaining_data = np.array([X_train_full_normalized[i] for i in remaining_indices]).astype('float32')
    index_flat = faiss.IndexFlatL2(embedding_dim)  # Initialize FAISS index for clustering
    clustering.train(remaining_data, index_flat)
    
    # Get cluster assignments
    D, cluster_assignments = index_flat.search(remaining_data, 1)
    
    # Convert FAISS centroids to numpy array
    centroids = faiss.vector_to_array(clustering.centroids).reshape(num_clusters, embedding_dim)

    # Select samples from each cluster using a hybrid strategy
    selected_indices = []
    for cluster in range(num_clusters):
        cluster_indices = [i for i, label in zip(remaining_indices, cluster_assignments) if label == cluster]
        if cluster_indices:
            # Find the closest sample to the cluster center
            cluster_center = centroids[cluster].reshape(1, -1)
            distances, _ = index.search(cluster_center, len(cluster_indices))
            closest_sample_index = cluster_indices[distances[0].argmin()]
            selected_indices.append(closest_sample_index)

            # Diversity Sampling: Select most diverse samples in the cluster
            cluster_data = X_train_full_normalized[cluster_indices]
            pairwise_distances_matrix = pairwise_distances(cluster_data)
            diversity_scores = pairwise_distances_matrix.mean(axis=1)
            most_diverse_index = cluster_indices[np.argmax(diversity_scores)]
            selected_indices.append(most_diverse_index)

    # Uncertainty sampling
    if len(remaining_indices) > 0:
        # Predict probabilities for remaining samples
        probs = clf.predict_proba(X_train_full_normalized[remaining_indices])
        # Calculate uncertainty as 1 - max probability
        uncertainty = 1 - np.max(probs, axis=1)
        
        # Determine the number of uncertain samples to select
        num_uncertain_samples = int(sample_size * uncertainty_threshold)
        if num_uncertain_samples > len(remaining_indices):
            num_uncertain_samples = len(remaining_indices)
        
        # Select the most uncertain samples
        uncertain_indices = np.argsort(-uncertainty)[:num_uncertain_samples]
        
        # Add uncertain samples to selected indices
        selected_indices.extend([remaining_indices[i] for i in uncertain_indices])

    # Ensure selected indices are unique and limited to the budget per iteration
    selected_indices = list(set(selected_indices))
    if len(selected_indices) > budget_per_iteration:
        selected_indices = selected_indices[:budget_per_iteration]

    # Add selected samples to the training set
    X_train = np.vstack((X_train, X_train_full_normalized[selected_indices]))
    y_train = np.concatenate((y_train, np.array(y_train_full)[selected_indices]))
    #print(np.unique(y_train))

    # Remove selected samples from the pool
    remaining_indices = list(set(remaining_indices) - set(selected_indices))

    # Update FAISS Index with new training data
    index.add(X_train_full_normalized[selected_indices])

    # Incrementally train the classifier with new samples
    clf.partial_fit(X_train, y_train)

    # Evaluate the classifier on the test set after each iteration
    y_pred = clf.predict(X_test_normalized)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Iteration {iteration + 1}: Test Accuracy = {accuracy:.4f}")

Iteration 1: Test Accuracy = 0.5221
Iteration 2: Test Accuracy = 0.5272
Iteration 3: Test Accuracy = 0.5847
Iteration 4: Test Accuracy = 0.4927
Iteration 5: Test Accuracy = 0.5995


KeyboardInterrupt: 

In [45]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.57      0.19      0.29       134
           1       0.58      0.15      0.24        47
           2       0.00      0.00      0.00        19
           3       0.00      0.00      0.00        10
           4       0.62      0.47      0.53      1371
           5       0.00      0.00      0.00        35
           6       0.61      0.88      0.72      2233
           7       1.00      0.02      0.04        53
           8       0.00      0.00      0.00        15
           9       0.00      0.00      0.00         5
          10       0.73      0.50      0.59       406
          11       0.00      0.00      0.00         5
          12       0.00      0.00      0.00        16
          13       0.00      0.00      0.00        23
          14       0.00      0.00      0.00        80
          15       0.71      0.40      0.51        55
          16       0.00      0.00      0.00         3
          17       0.32    

/home/student/.cache/pypoetry/virtualenvs/dataanalysisvisualizationfiles-qa5NmZKI-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/student/.cache/pypoetry/virtualenvs/dataanalysisvisualizationfiles-qa5NmZKI-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/student/.cache/pypoetry/virtualenvs/dataanalysisvisualizationfiles-qa5NmZKI-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined an

### ANN As Clustering 2

In [83]:
np.unique(y_train)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [31]:
minority_focus_ratio = 0.3 
dynamic_clusters = 5
# Define a threshold to identify minority classes
minority_threshold = 100  # You can adjust this threshold based on your dataset

# Calculate the original class distribution
original_class_distribution = np.bincount(y_train_full)

# Identify minority classes based on the threshold
minority_classes = [i for i, count in enumerate(original_class_distribution) if count < minority_threshold]
# Initialize the dictionary to store class distribution for each cluster
cluster_class_distribution = {i: {} for i in range(num_clusters)}

In [62]:
original_class_distribution = np.bincount(y_train_full)
print("Original Class Distribution:")
for class_label, count in enumerate(original_class_distribution):
    print(f"Class {class_label}: {count} samples")

Original Class Distribution:
Class 0: 539 samples
Class 1: 164 samples
Class 2: 71 samples
Class 3: 38 samples
Class 4: 5314 samples
Class 5: 106 samples
Class 6: 9081 samples
Class 7: 210 samples
Class 8: 56 samples
Class 9: 21 samples
Class 10: 1753 samples
Class 11: 25 samples
Class 12: 97 samples
Class 13: 105 samples
Class 14: 299 samples
Class 15: 189 samples
Class 16: 12 samples
Class 17: 895 samples
Class 18: 41 samples
Class 19: 466 samples


In [32]:
# Active Learning Loop
for iteration in range(iterations):
    # Adjust the number of clusters dynamically
    
    num_clusters = dynamic_clusters + iteration  # Increase clusters gradually

    # Initialize FAISS clustering
    clustering = faiss.Clustering(embedding_dim, num_clusters)
    clustering.verbose = False
    clustering.niter = 50

    # Convert remaining indices to the appropriate format
    remaining_data = np.array([X_train_full_normalized[i] for i in remaining_indices]).astype('float32')
    index_flat = faiss.IndexFlatL2(embedding_dim)
    clustering.train(remaining_data, index_flat)

    # Get cluster assignments
    D, cluster_assignments = index_flat.search(remaining_data, 1)
    
    # Convert FAISS centroids to numpy array
    centroids = faiss.vector_to_array(clustering.centroids).reshape(num_clusters, embedding_dim)
    
    # Print class distribution within each cluster
    # print("\nClass distribution within each cluster:")
    # for cluster in range(num_clusters):
    #     cluster_indices = [i for i, label in zip(remaining_indices, cluster_assignments) if label == cluster]
    #     if cluster_indices:
    #         cluster_labels = [y_train_full[i] for i in cluster_indices]
    #         unique, counts = np.unique(cluster_labels, return_counts=True)
    #         cluster_distribution = dict(zip(unique, counts))
    #         print(f"Cluster {cluster}: {cluster_distribution}")

    # Select samples from each cluster using a refined strategy
    selected_indices = []
    for cluster in range(num_clusters):
        cluster_indices = [i for i, label in zip(remaining_indices, cluster_assignments) if label == cluster]
        if cluster_indices:
            # Select the most uncertain samples within each cluster
            cluster_data = np.array([X_train_full_normalized[i] for i in cluster_indices]).astype('float32')
            probs = clf.predict_proba(cluster_data)
            uncertainty = 1 - np.max(probs, axis=1)

            # Calculate diversity by distance from the centroid
            distances_from_centroid = np.linalg.norm(cluster_data - centroids[cluster], axis=1)
            combined_scores = uncertainty + distances_from_centroid

            # Prioritize samples with high uncertainty and diversity
            sorted_indices = np.argsort(-combined_scores)
            num_samples = min(int(len(cluster_indices) * uncertainty_threshold), len(cluster_indices))
            selected_cluster_indices = [cluster_indices[i] for i in sorted_indices[:num_samples]]
            selected_indices.extend(selected_cluster_indices)

    # Ensure unique samples and limit to the budget
    selected_indices = list(set(selected_indices))
    if len(selected_indices) > budget_per_iteration:
        selected_indices = selected_indices[:budget_per_iteration]

    # Add selected samples to the training set
    X_train = np.vstack((X_train, X_train_full_normalized[selected_indices]))
    y_train = np.concatenate((y_train, np.array(y_train_full)[selected_indices]))

    # Remove selected samples from the pool
    remaining_indices = list(set(remaining_indices) - set(selected_indices))

    # Update FAISS Index with new training data
    index.add(X_train_full_normalized[selected_indices])

    # Incrementally train the classifier with new samples
    clf.partial_fit(X_train, y_train)

    # Evaluate the classifier on the test set after each iteration
    y_pred = clf.predict(X_test_normalized)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Iteration {iteration + 1}: Test Accuracy = {accuracy:.4f}")

Iteration 1: Test Accuracy = 0.4931
Iteration 2: Test Accuracy = 0.5204
Iteration 3: Test Accuracy = 0.6009
Iteration 4: Test Accuracy = 0.5878
Iteration 5: Test Accuracy = 0.5968
Iteration 6: Test Accuracy = 0.5939
Iteration 7: Test Accuracy = 0.6335
Iteration 8: Test Accuracy = 0.6243
Iteration 9: Test Accuracy = 0.6325
Iteration 10: Test Accuracy = 0.6370
Iteration 11: Test Accuracy = 0.6216
Iteration 12: Test Accuracy = 0.6405
Iteration 13: Test Accuracy = 0.6112
Iteration 14: Test Accuracy = 0.6416
Iteration 15: Test Accuracy = 0.6483
Iteration 16: Test Accuracy = 0.6455
Iteration 17: Test Accuracy = 0.6413
Iteration 18: Test Accuracy = 0.6465
Iteration 19: Test Accuracy = 0.6463
Iteration 20: Test Accuracy = 0.6446


In [33]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.53      0.35      0.42       134
           1       0.57      0.09      0.15        47
           2       0.00      0.00      0.00        19
           3       0.00      0.00      0.00        10
           4       0.58      0.69      0.63      1371
           5       0.00      0.00      0.00        35
           6       0.69      0.78      0.73      2233
           7       0.67      0.04      0.07        53
           8       0.00      0.00      0.00        15
           9       0.00      0.00      0.00         5
          10       0.65      0.67      0.66       406
          11       0.00      0.00      0.00         5
          12       0.00      0.00      0.00        16
          13       0.00      0.00      0.00        23
          14       0.33      0.01      0.02        80
          15       0.89      0.29      0.44        55
          16       0.00      0.00      0.00         3
          17       0.29    

/home/student/.cache/pypoetry/virtualenvs/dataanalysisvisualizationfiles-qa5NmZKI-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/student/.cache/pypoetry/virtualenvs/dataanalysisvisualizationfiles-qa5NmZKI-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/student/.cache/pypoetry/virtualenvs/dataanalysisvisualizationfiles-qa5NmZKI-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined an

## NN Model

In [98]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

In [99]:
class NN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super (NN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [109]:
# Define device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
input_size = X_train_full_normalized.shape[1]
hidden_size = 128  # You can adjust the number of neurons
num_classes = len(np.unique(y_train_full))
learning_rate = 0.001

# Initialize the neural network, loss function, and optimizer
model = NN(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [108]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(X_test_normalized, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)


In [102]:
def train_nn(model, X_train, y_train, optimizer, criterion, batch_size=64, epochs=5):
    model.train()

    # Create dataset and dataloader
    dataset = TensorDataset(X_train, y_train)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in loader:
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(loader):.4f}')

In [103]:
def evaluate_nn(model, X_test, batch_size=64):
    model.eval()

    # Convert data to PyTorch tensor
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    
    # Create dataset and dataloader
    dataset = TensorDataset(X_test_tensor)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    predictions = []
    with torch.no_grad():
        for inputs in loader:
            outputs = model(inputs[0])
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
    return np.array(predictions)

In [104]:
# Active Learning Loop
for iteration in range(iterations):
    # Adjust the number of clusters dynamically
    num_clusters = min(dynamic_clusters + iteration, len(remaining_indices) // 2)

    # Initialize FAISS clustering
    clustering = faiss.Clustering(embedding_dim, num_clusters)
    clustering.verbose = False
    clustering.niter = 50

    # Convert remaining indices to the appropriate format
    remaining_data = np.array([X_train_full_normalized[i] for i in remaining_indices]).astype('float32')
    index_flat = faiss.IndexFlatL2(embedding_dim)
    clustering.train(remaining_data, index_flat)

    # Get cluster assignments
    _, cluster_assignments = index_flat.search(remaining_data, 1)
    cluster_assignments = cluster_assignments.flatten()

    # Convert FAISS centroids to numpy array
    centroids = faiss.vector_to_array(clustering.centroids).reshape(num_clusters, embedding_dim)

    # Select samples from each cluster using a refined strategy
    selected_indices = []
    for cluster in range(num_clusters):
        cluster_indices = [i for i, label in zip(remaining_indices, cluster_assignments) if label == cluster]
        if cluster_indices:
            # Select the most uncertain samples within each cluster
            cluster_data = np.array([X_train_full_normalized[i] for i in cluster_indices]).astype('float32')
            cluster_data_tensor = torch.tensor(cluster_data, dtype=torch.float32).to(device)

            # Get model predictions and uncertainties
            model.eval()
            with torch.no_grad():
                outputs = model(cluster_data_tensor)
                probs = nn.functional.softmax(outputs, dim=1)
                uncertainty = 1 - torch.max(probs, dim=1).values.cpu().numpy()

            # Calculate diversity by distance from the centroid
            distances_from_centroid = np.linalg.norm(cluster_data - centroids[cluster], axis=1)
            combined_scores = uncertainty + distances_from_centroid

            # Prioritize samples with high uncertainty and diversity
            sorted_indices = np.argsort(-combined_scores)
            num_samples = min(int(len(cluster_indices) * uncertainty_threshold), len(cluster_indices))
            selected_cluster_indices = [cluster_indices[i] for i in sorted_indices[:num_samples]]
            selected_indices.extend(selected_cluster_indices)

    # Ensure unique samples and limit to the budget
    selected_indices = list(set(selected_indices))
    if len(selected_indices) > budget_per_iteration:
        selected_indices = selected_indices[:budget_per_iteration]

    # Add selected samples to the training set
    X_train = np.vstack((X_train, X_train_full_normalized[selected_indices]))
    y_train = np.concatenate((y_train, np.array(y_train_full)[selected_indices]))

    # Remove selected samples from the pool
    remaining_indices = list(set(remaining_indices) - set(selected_indices))

    # Update FAISS Index with new training data
    index_flat.add(X_train_full_normalized[selected_indices])

    # Convert updated training data to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

    # Incrementally train the neural network with the new samples
    train_nn(model, X_train_tensor, y_train_tensor, optimizer, criterion, batch_size=32, epochs=5)

    # Evaluate the neural network on the test set after each iteration
    X_test_tensor = torch.tensor(X_test_normalized, dtype=torch.float32).to(device)
    y_pred = evaluate_nn(model, X_test_tensor)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Iteration {iteration + 1}: Test Accuracy = {accuracy:.4f}")


WARNING clustering 14 points to 5 centroids: please provide at least 195 training points


Epoch [1/5], Loss: 2.9518
Epoch [2/5], Loss: 2.9193
Epoch [3/5], Loss: 2.8840
Epoch [4/5], Loss: 2.8416
Epoch [5/5], Loss: 2.8002
Iteration 1: Test Accuracy = 0.3500


/tmp/ipykernel_5349/1057497763.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
WARNING clustering 14 points to 6 centroids: please provide at least 234 training points


Epoch [1/5], Loss: 2.7380
Epoch [2/5], Loss: 2.6956
Epoch [3/5], Loss: 2.6078
Epoch [4/5], Loss: 2.5587
Epoch [5/5], Loss: 2.4624
Iteration 2: Test Accuracy = 0.3745


/tmp/ipykernel_5349/1057497763.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
WARNING clustering 14 points to 7 centroids: please provide at least 273 training points


...

/tmp/ipykernel_5349/1057497763.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
WARNING clustering 14 points to 7 centroids: please provide at least 273 training points


Epoch [1/5], Loss: 0.1051
Epoch [2/5], Loss: 0.1019
Epoch [3/5], Loss: 0.0967
Epoch [4/5], Loss: 0.0986
Epoch [5/5], Loss: 0.0923
Iteration 18: Test Accuracy = 0.5309


/tmp/ipykernel_5349/1057497763.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
WARNING clustering 14 points to 7 centroids: please provide at least 273 training points


Epoch [1/5], Loss: 0.0934
Epoch [2/5], Loss: 0.0887
Epoch [3/5], Loss: 0.0801
Epoch [4/5], Loss: 0.0792
Epoch [5/5], Loss: 0.0731
Iteration 19: Test Accuracy = 0.5332
Epoch [1/5], Loss: 0.0720
Epoch [2/5], Loss: 0.0711
Epoch [3/5], Loss: 0.0700
Epoch [4/5], Loss: 0.0684
Epoch [5/5], Loss: 0.0666
Iteration 20: Test Accuracy = 0.5325


/tmp/ipykernel_5349/1057497763.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
WARNING clustering 14 points to 7 centroids: please provide at least 273 training points
/tmp/ipykernel_5349/1057497763.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)


In [105]:
y_pred_final = evaluate_nn(model, X_test_normalized)
final_accuracy = accuracy_score(y_test, y_pred_final)
print(f"Final Test Accuracy after all iterations: {final_accuracy:.4f}")

Final Test Accuracy after all iterations: 0.5325


In [111]:
# Active Learning Loop
accuracy_scores = []

for iteration in range(iterations):
    # Incrementally train the neural network with the current training set
    train_nn(model, X_train_tensor, y_train_tensor, optimizer, criterion, batch_size=32, epochs=5)
    
    # Predict and evaluate performance on the test set
    y_pred = evaluate_nn(model, X_test_tensor)
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    
    print(f"Iteration {iteration + 1}: Accuracy = {accuracy_scores[-1]:.4f}")

    # Predict probabilities on the pool data to measure uncertainty
    model.eval()
    with torch.no_grad():
        X_pool_tensor = torch.tensor(X_pool, dtype=torch.float32).to(device)
        pool_probs = nn.functional.softmax(model(X_pool_tensor), dim=1)
        uncertainty = 1 - torch.max(pool_probs, dim=1).values.cpu().numpy()

    # Select samples based on uncertainty
    sample_size = min(sample_size, len(X_pool))
    selected_indices = np.argsort(-uncertainty)[:sample_size]
    X_sample = X_pool[selected_indices]
    y_sample = y_pool[selected_indices]

    # Update the training set with new samples
    X_train = np.vstack([X_train, X_sample])
    y_train = np.concatenate([y_train, y_sample])

    # Convert updated training data to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

    # Remove the selected samples from the pool
    X_pool = np.delete(X_pool, selected_indices, axis=0)
    y_pool = np.delete(y_pool, selected_indices, axis=0)

    # Stop criteria based on the pool size or maximum training size
    if len(X_pool) == 0 or len(X_pool) < sample_size or len(X_train) >= 60000:
        break

# Print the final classification report
y_pred_final = evaluate_nn(model, X_test_tensor)
print(classification_report(y_test, y_pred_final))

Epoch [1/5], Loss: 2.9887
Epoch [2/5], Loss: 2.9415
Epoch [3/5], Loss: 2.9004
Epoch [4/5], Loss: 2.8694
Epoch [5/5], Loss: 2.8125
Iteration 1: Accuracy = 0.5182
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       134
           1       0.00      0.00      0.00        47
           2       0.00      0.00      0.00        19
           3       0.00      0.00      0.00        10
           4       0.49      0.52      0.50      1371
           5       0.00      0.00      0.00        35
           6       0.53      0.81      0.64      2233
           7       0.00      0.00      0.00        53
           8       0.00      0.00      0.00        15
           9       0.00      0.00      0.00         5
          10       0.00      0.00      0.00       406
          11       0.00      0.00      0.00         5
          12       0.00      0.00      0.00        16
          13       0.00      0.00      0.00        23
          14       0.00     

/tmp/ipykernel_5349/1057497763.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
/tmp/ipykernel_5349/1057497763.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
/home/student/.cache/pypoetry/virtualenvs/dataanalysisvisualizationfiles-qa5NmZKI-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result